# Feature extraction for time series 

In [53]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
sys.path.insert(1, module_path)
import utility
import csv  

import pandas as pd
import time
import random
from PyEMD import EEMD
from PyEMD import EMD
import librosa
import numpy as np
import pywt
from scipy.stats import skew 

In [56]:
N_IMFS = 10
N_SIFTS = 15
SELECT_IMFS = [4,5,6,7]
WAVELET = 'db8'
LEVELS_DWT = 10
N_SAMPLES = 2
TARGET_RATE = 40000
DECOMP = 'EEMD_DWT'

def get_n_best_IMFs(timeseries, sr, method = 'EMD'):
    m_trials = 2
    result = []
    t = utility.get_t(timeseries, sr)
    if method == 'EEMD':
        eemd = EEMD(trials=m_trials)
        eemd.spline_kind="slinear"
        eemd.FIXE = N_SIFTS
        # Execute EEMD on S
        IMFs = eemd.eemd(timeseries, t, max_imf = N_IMFS)
    else:
        emd = EMD()
        emd.spline_kind="slinear"
        emd.FIXE = N_SIFTS
        # Execute EEMD on S
        IMFs = emd.emd(timeseries, t, max_imf = N_IMFS)
    for idx in range(len(IMFs)):
        if idx in SELECT_IMFS:
            result.append(IMFs[idx])

    return result

def get_n_best_levels(timeseries):
    dwtValues = pywt.wavedec(timeseries, WAVELET, level=LEVELS_DWT)
    len_D = len(dwtValues)
    result = []
    for n, c_D in enumerate(dwtValues):
        if n == 0:
            continue
        result.append(c_D)
    return result

def get_entropy(timeseries):
    timeseries_nz = timeseries[timeseries != 0]
    return - np.sum(((timeseries_nz**2)*np.log(timeseries_nz**2)))
    
def get_energy(timeseries):  
    N = len(timeseries)
    return np.sum(np.abs(timeseries) ** 2) / N
    

def get_features(timeseries, sr):
    ft2 = librosa.feature.zero_crossing_rate(timeseries)[0]
    ft3 = librosa.feature.spectral_rolloff(timeseries)[0]
    ft4 = librosa.feature.spectral_centroid(timeseries)[0]
    ft5 = librosa.feature.spectral_contrast(timeseries)[0]
    ft6 = librosa.feature.spectral_bandwidth(timeseries)[0]

    ### Get HOS and simple features 
    ft0_trunc = np.hstack((np.mean(timeseries), np.std(timeseries),
                           skew(timeseries), np.max(timeseries),
                           np.median(timeseries), np.min(timeseries),
                           get_energy(timeseries), get_entropy(timeseries)))
  
    ### Spectral Features 
    ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.median(ft2), np.min(ft2)))
    ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.median(ft3), np.min(ft3)))
    ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.median(ft4), np.min(ft4)))
    ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.median(ft5), np.min(ft5)))
    ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.median(ft6), np.max(ft6)))
    return np.hstack((ft0_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc))


def get_row(timeseries, sr):
    if DECOMP == 'noDecomp':
        return get_features(timeseries,sr)
    elif DECOMP in ['EMD','EEMD']:
        IMFS = get_n_best_IMFs(timeseries, sr, method = DECOMP)
        all_features = get_features(IMFS[0],sr)
        for n in range(1,len(IMFS)):
            print(all_features)
            all_features = np.concatenate([all_features, get_features(IMFS[n],sr)])
        return all_features
    elif DECOMP == 'DWT':
        levels = get_n_best_levels(timeseries)
        all_features = get_features(levels[0],sr)
        for n in range(1,len(levels)):
            print(all_features)
            all_features = np.concatenate([all_features, get_features(levels[n],sr)])
        return all_features
    elif DECOMP in ['EMD_DWT', 'EEMD_DWT']:
        method = 'EMD'
        if DECOMP == 'EEMD_DWT': method = 'EEMD'
        imf_levels = []
        
        IMFS = get_n_best_IMFs(timeseries, sr, method = method)
        for imf in IMFS:
            levels = get_n_best_levels(imf)
            for level in levels:
                imf_levels.append(level)
        
        all_features = get_features(imf_levels[0],sr)
        for n in range(1,len(imf_levels)):
            print(all_features)
            all_features = np.concatenate([all_features, get_features(imf_levels[n],sr)])
                
        return all_features
    return False


def add_row_csv(timeseries, sr, label, filename, csv_filename):
        row = get_row(timeseries, sr)
        row = np.concatenate([row.astype(str), np.array([filename, label])])
        if os.path.exists(csv_filename):
            with open(csv_filename, 'a') as f:
                writer = csv.writer(f)
                writer.writerow(row)
        else:
            with open(csv_filename, 'w') as f:
                writer = csv.writer(f)
                writer.writerow(row)

In [10]:
def read_audio_write_features(path, label, filename, csv_filename):
        sr, y = utility.read_wav_file(path, TARGET_RATE)
        y = utility.denoise_audio(y)
        y, sr = utility.downsample(y, sr)
        add_row_csv(y, sr, label, filename, csv_filename)

def get_dataset(filepath, labels, csv_filename):
    '''
    Creating a dataset, specifically for audio-files
    '''
    random.seed(20)
    filepath1 = filepath + '/' + labels[0]
    filepath2 = filepath + '/' + labels[1]
    # Assumes that the data beloning to a label is saved in seperate directories 
    label1 = os.listdir(filepath1) 
    label2 = os.listdir(filepath2)
    
    label1_idx = random.sample(range(len(label1)), N_SAMPLES)
    label2_idx = random.sample(range(len(label1)), N_SAMPLES)
    
    for i in range(0,N_SAMPLES):
        print(f'Processing : sample number {i} of {N_SAMPLES}')
        filename1 , filename2 =  label1[label1_idx[i]], label2[label2_idx[i]]
        read_audio_write_features(filepath1 + '/' + filename1, labels[0], filename1, csv_filename)
        read_audio_write_features(filepath2 + '/' +  filename2, labels[1],filename2, csv_filename)
          

In [65]:
start = time.time()
filepath = '../../data'
labels = ['crackle','no-crackle']
get_dataset(filepath, labels, DECOMP+'.csv')
print(f' Processing finished, total time used = {time.time() - start}')


Processing : sample number 0 of 2
[-3.93594026e-03  8.98384772e-02 -6.20613420e-01  2.63393010e-01
 -1.18636137e-03 -3.57772640e-01  8.08644362e-03  9.90474852e-01
  1.17187500e-02  0.00000000e+00  0.00000000e+00  1.17187500e-02
  1.17187500e-02  1.17187500e-02  1.04651367e+04  0.00000000e+00
  0.00000000e+00  1.04651367e+04  1.04651367e+04  1.04651367e+04
  6.39288375e+03  0.00000000e+00  0.00000000e+00  6.39288375e+03
  6.39288375e+03  6.39288375e+03  4.52860119e+01  0.00000000e+00
  0.00000000e+00  4.52860119e+01  4.52860119e+01  4.52860119e+01
  3.26812666e+03  0.00000000e+00  0.00000000e+00  3.26812666e+03
  3.26812666e+03  3.26812666e+03]
[-3.93594026e-03  8.98384772e-02 -6.20613420e-01  2.63393010e-01
 -1.18636137e-03 -3.57772640e-01  8.08644362e-03  9.90474852e-01
  1.17187500e-02  0.00000000e+00  0.00000000e+00  1.17187500e-02
  1.17187500e-02  1.17187500e-02  1.04651367e+04  0.00000000e+00
  0.00000000e+00  1.04651367e+04  1.04651367e+04  1.04651367e+04
  6.39288375e+03  0.00

[-3.93594026e-03  8.98384772e-02 -6.20613420e-01  2.63393010e-01
 -1.18636137e-03 -3.57772640e-01  8.08644362e-03  9.90474852e-01
  1.17187500e-02  0.00000000e+00  0.00000000e+00  1.17187500e-02
  1.17187500e-02  1.17187500e-02  1.04651367e+04  0.00000000e+00
  0.00000000e+00  1.04651367e+04  1.04651367e+04  1.04651367e+04
  6.39288375e+03  0.00000000e+00  0.00000000e+00  6.39288375e+03
  6.39288375e+03  6.39288375e+03  4.52860119e+01  0.00000000e+00
  0.00000000e+00  4.52860119e+01  4.52860119e+01  4.52860119e+01
  3.26812666e+03  0.00000000e+00  0.00000000e+00  3.26812666e+03
  3.26812666e+03  3.26812666e+03 -6.36226382e-04  9.79993019e-02
  6.28886393e-01  4.11565939e-01 -9.70880146e-04 -3.40876853e-01
  9.60426796e-03  1.80020680e+00  1.66015625e-02  0.00000000e+00
  0.00000000e+00  1.66015625e-02  1.66015625e-02  1.66015625e-02
  1.01744385e+04  0.00000000e+00  0.00000000e+00  1.01744385e+04
  1.01744385e+04  1.01744385e+04  6.65359272e+03  0.00000000e+00
  0.00000000e+00  6.65359

[-3.93594026e-03  8.98384772e-02 -6.20613420e-01  2.63393010e-01
 -1.18636137e-03 -3.57772640e-01  8.08644362e-03  9.90474852e-01
  1.17187500e-02  0.00000000e+00  0.00000000e+00  1.17187500e-02
  1.17187500e-02  1.17187500e-02  1.04651367e+04  0.00000000e+00
  0.00000000e+00  1.04651367e+04  1.04651367e+04  1.04651367e+04
  6.39288375e+03  0.00000000e+00  0.00000000e+00  6.39288375e+03
  6.39288375e+03  6.39288375e+03  4.52860119e+01  0.00000000e+00
  0.00000000e+00  4.52860119e+01  4.52860119e+01  4.52860119e+01
  3.26812666e+03  0.00000000e+00  0.00000000e+00  3.26812666e+03
  3.26812666e+03  3.26812666e+03 -6.36226382e-04  9.79993019e-02
  6.28886393e-01  4.11565939e-01 -9.70880146e-04 -3.40876853e-01
  9.60426796e-03  1.80020680e+00  1.66015625e-02  0.00000000e+00
  0.00000000e+00  1.66015625e-02  1.66015625e-02  1.66015625e-02
  1.01744385e+04  0.00000000e+00  0.00000000e+00  1.01744385e+04
  1.01744385e+04  1.01744385e+04  6.65359272e+03  0.00000000e+00
  0.00000000e+00  6.65359

[-3.93594026e-03  8.98384772e-02 -6.20613420e-01 ...  3.06098691e+03
  3.06098691e+03  3.06098691e+03]
[-3.93594026e-03  8.98384772e-02 -6.20613420e-01 ...  2.45694666e+03
  2.45694666e+03  2.45694666e+03]
[-3.93594026e-03  8.98384772e-02 -6.20613420e-01 ...  2.19729537e+03
  2.19729537e+03  2.19729537e+03]
[-3.93594026e-03  8.98384772e-02 -6.20613420e-01 ...  2.38677200e+03
  2.38312557e+03  2.38677200e+03]
[-3.93594026e-03  8.98384772e-02 -6.20613420e-01 ...  2.40795575e+03
  2.29031510e+03  2.40795575e+03]
[-3.93594026e-03  8.98384772e-02 -6.20613420e-01 ...  2.66424535e+03
  2.29585317e+03  2.66424535e+03]
[-3.93594026e-03  8.98384772e-02 -6.20613420e-01 ...  2.61816176e+03
  2.26659364e+03  2.61816176e+03]


/home/andrine/anaconda3/lib/python3.7/site-packages/pywt/_multilevel.py:45: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


...

[-1.99234852e-03  2.17457132e-01 -1.13319638e+00  5.01896858e-01
 -9.81226140e-04 -7.45705128e-01  4.72915738e-02  2.07900720e+00
  8.78906250e-03  0.00000000e+00  0.00000000e+00  8.78906250e-03
  8.78906250e-03  8.78906250e-03  9.64687500e+03  0.00000000e+00
  0.00000000e+00  9.64687500e+03  9.64687500e+03  9.64687500e+03
  6.44212744e+03  0.00000000e+00  0.00000000e+00  6.44212744e+03
  6.44212744e+03  6.44212744e+03  1.05979457e+02  0.00000000e+00
  0.00000000e+00  1.05979457e+02  1.05979457e+02  1.05979457e+02
  2.89804352e+03  0.00000000e+00  0.00000000e+00  2.89804352e+03
  2.89804352e+03  2.89804352e+03 -1.27629276e-02  1.02162604e-01
 -7.35794331e-01  3.11936093e-01 -1.17350559e-02 -4.41615072e-01
  1.06000899e-02  1.58611576e+00  1.17187500e-02  0.00000000e+00
  0.00000000e+00  1.17187500e-02  1.17187500e-02  1.17187500e-02
  9.74377441e+03  0.00000000e+00  0.00000000e+00  9.74377441e+03
  9.74377441e+03  9.74377441e+03  6.23094354e+03  0.00000000e+00
  0.00000000e+00  6.23094

[-1.99234852e-03  2.17457132e-01 -1.13319638e+00  5.01896858e-01
 -9.81226140e-04 -7.45705128e-01  4.72915738e-02  2.07900720e+00
  8.78906250e-03  0.00000000e+00  0.00000000e+00  8.78906250e-03
  8.78906250e-03  8.78906250e-03  9.64687500e+03  0.00000000e+00
  0.00000000e+00  9.64687500e+03  9.64687500e+03  9.64687500e+03
  6.44212744e+03  0.00000000e+00  0.00000000e+00  6.44212744e+03
  6.44212744e+03  6.44212744e+03  1.05979457e+02  0.00000000e+00
  0.00000000e+00  1.05979457e+02  1.05979457e+02  1.05979457e+02
  2.89804352e+03  0.00000000e+00  0.00000000e+00  2.89804352e+03
  2.89804352e+03  2.89804352e+03 -1.27629276e-02  1.02162604e-01
 -7.35794331e-01  3.11936093e-01 -1.17350559e-02 -4.41615072e-01
  1.06000899e-02  1.58611576e+00  1.17187500e-02  0.00000000e+00
  0.00000000e+00  1.17187500e-02  1.17187500e-02  1.17187500e-02
  9.74377441e+03  0.00000000e+00  0.00000000e+00  9.74377441e+03
  9.74377441e+03  9.74377441e+03  6.23094354e+03  0.00000000e+00
  0.00000000e+00  6.23094

[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  3.49460169e+03
  3.49460169e+03  3.49460169e+03]
[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  3.15741221e+03
  3.15741221e+03  3.15741221e+03]
[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  2.88438169e+03
  2.88438169e+03  2.88438169e+03]
[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  2.56434063e+03
  2.56434063e+03  2.56434063e+03]
[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  2.11643960e+03
  2.11643960e+03  2.11643960e+03]
[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  2.41227474e+03
  2.37992533e+03  2.41227474e+03]
[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  2.88502626e+03
  2.41897708e+03  2.88502626e+03]
[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  2.38719886e+03
  2.33599493e+03  2.38719886e+03]
[-1.99234852e-03  2.17457132e-01 -1.13319638e+00 ...  3.25099048e+03
  2.34235826e+03  3.25099048e+03]
[-1.88760848e-03  7.60771183e-02 -9.16153882e-02  1.29881166e-01
 -7.8618

[-1.88760848e-03  7.60771183e-02 -9.16153882e-02  1.29881166e-01
 -7.86185530e-03 -1.73213317e-01  5.79129100e-03  5.63678989e-01
  4.39453125e-03  0.00000000e+00  0.00000000e+00  4.39453125e-03
  4.39453125e-03  4.39453125e-03  9.96987305e+03  0.00000000e+00
  0.00000000e+00  9.96987305e+03  9.96987305e+03  9.96987305e+03
  6.20575074e+03  0.00000000e+00  0.00000000e+00  6.20575074e+03
  6.20575074e+03  6.20575074e+03  4.55011557e+01  0.00000000e+00
  0.00000000e+00  4.55011557e+01  4.55011557e+01  4.55011557e+01
  2.67522765e+03  0.00000000e+00  0.00000000e+00  2.67522765e+03
  2.67522765e+03  2.67522765e+03 -1.49429853e-03  7.08454625e-02
  1.20572417e-01  1.64458953e-01 -8.61577265e-03 -1.97202872e-01
  5.02131249e-03  6.11467188e-01  9.27734375e-03  0.00000000e+00
  0.00000000e+00  9.27734375e-03  9.27734375e-03  9.27734375e-03
  9.05471191e+03  0.00000000e+00  0.00000000e+00  9.05471191e+03
  9.05471191e+03  9.05471191e+03  5.24245036e+03  0.00000000e+00
  0.00000000e+00  5.24245

[-1.88760848e-03  7.60771183e-02 -9.16153882e-02  1.29881166e-01
 -7.86185530e-03 -1.73213317e-01  5.79129100e-03  5.63678989e-01
  4.39453125e-03  0.00000000e+00  0.00000000e+00  4.39453125e-03
  4.39453125e-03  4.39453125e-03  9.96987305e+03  0.00000000e+00
  0.00000000e+00  9.96987305e+03  9.96987305e+03  9.96987305e+03
  6.20575074e+03  0.00000000e+00  0.00000000e+00  6.20575074e+03
  6.20575074e+03  6.20575074e+03  4.55011557e+01  0.00000000e+00
  0.00000000e+00  4.55011557e+01  4.55011557e+01  4.55011557e+01
  2.67522765e+03  0.00000000e+00  0.00000000e+00  2.67522765e+03
  2.67522765e+03  2.67522765e+03 -1.49429853e-03  7.08454625e-02
  1.20572417e-01  1.64458953e-01 -8.61577265e-03 -1.97202872e-01
  5.02131249e-03  6.11467188e-01  9.27734375e-03  0.00000000e+00
  0.00000000e+00  9.27734375e-03  9.27734375e-03  9.27734375e-03
  9.05471191e+03  0.00000000e+00  0.00000000e+00  9.05471191e+03
  9.05471191e+03  9.05471191e+03  5.24245036e+03  0.00000000e+00
  0.00000000e+00  5.24245